In [8]:
!pip install transformers[torch]


In [9]:
pip install accelerate -U


  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/314.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/314.1 kB ? eta -:--:--
   -------------------- ------------------- 163.8/314.1 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 314.1/314.1 kB 3.9 MB/s eta 0:00:00
Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
Using cached tbb-2021.13.0-py3-none-win_amd64.whl (286 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd

# Load datasets
clean_sampleset2000 = pd.read_csv('C:/Users/Devanand J/Desktop/Final/Yelp_dataset.csv')
restaurants_train2021 = pd.read_csv('C:/Users/Devanand J/Desktop/Final/SemEval.csv')
clean_sampleset2000['overall_polarity'] = clean_sampleset2000['overall_polarity'].apply(lambda x: 0 if x == 1 else 1)


In [2]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    clean_sampleset2000['preprocessed_comments'], clean_sampleset2000['overall_polarity'], test_size=0.2, random_state=42)


In [3]:
from transformers import GPT2TokenizerFast

# Initializing GPT for tokenization
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

# Padding steps
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=1024)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=1024)


C:\Users\Devanand J\anaconda3\envs\gpt_cuda118\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
test_dataset = SentimentDataset(test_encodings, test_labels.tolist())


In [5]:
from transformers import GPT2ForSequenceClassification, Trainer, TrainingArguments

# Loading GPT model
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id


C:\Users\Devanand J\anaconda3\envs\gpt_cuda118\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=4,   # Increased batch size for training
    per_device_eval_batch_size=8,    # Increased batch size for evaluation
    gradient_accumulation_steps=8,   # Accumulate gradients to simulate a larger batch size
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True                        # Enable mixed precision training
)


In [9]:
# Create Trainer instance
trainer = Trainer(
    model=model,                         
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset   
)

C:\Users\Devanand J\anaconda3\envs\gpt_cuda118\Lib\site-packages\accelerate\accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [8]:

trainer.train()


Epoch,Training Loss,Validation Loss
1,6.503300,6.358065
2,0.669500,0.662084
3,0.354400,0.407122


Checkpoint destination directory ./results\checkpoint-50 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results\checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results\checkpoint-150 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=150, training_loss=3.3615895668665567, metrics={'train_runtime': 24993.4421, 'train_samples_per_second': 0.192, 'train_steps_per_second': 0.006, 'total_flos': 2508448805683200.0, 'train_loss': 3.3615895668665567, 'epoch': 3.0})

In [9]:
# Saving the model
model.save_pretrained('./results/GPTsentiment_model')
tokenizer.save_pretrained('./results/GPTsentiment_model')

('./results/GPTsentiment_model\\tokenizer_config.json',
 './results/GPTsentiment_model\\special_tokens_map.json',
 './results/GPTsentiment_model\\vocab.json',
 './results/GPTsentiment_model\\merges.txt',
 './results/GPTsentiment_model\\added_tokens.json',
 './results/GPTsentiment_model\\tokenizer.json')

In [10]:
#from transformers import GPT2ForSequenceClassification, GPT2TokenizerFast

# Load the model
model = GPT2ForSequenceClassification.from_pretrained('./results/GPTsentiment_model')

# Load the tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained('./results/GPTsentiment_model')


In [11]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Predict on test set
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(test_labels, preds)
print(f"Accuracy: {accuracy}")

# Print classification report
report = classification_report(test_labels, preds, target_names=['negative', 'positive'])
print(report)


Evaluation results: {'eval_loss': 0.40712210536003113, 'eval_runtime': 45.5505, 'eval_samples_per_second': 8.781, 'eval_steps_per_second': 1.098}
Accuracy: 0.8275
              precision    recall  f1-score   support

    negative       0.95      0.69      0.80       201
    positive       0.76      0.96      0.85       199

    accuracy                           0.83       400
   macro avg       0.85      0.83      0.82       400
weighted avg       0.85      0.83      0.82       400



In [12]:
import torch
from tqdm import tqdm
import time

# Predict on the entire clean_sampleset2000 dataset with progress tracking
all_texts = clean_sampleset2000['preprocessed_comments'].tolist()
all_preds = []

batch_size = 16
num_batches = len(all_texts) // batch_size + (1 if len(all_texts) % batch_size != 0 else 0)

start_time = time.time()
for i in tqdm(range(num_batches), desc="Processing batches"):
    batch_texts = all_texts[i*batch_size:(i+1)*batch_size]
    batch_encodings = tokenizer(batch_texts, truncation=True, padding=True, max_length=1024)
    batch_dataset = SentimentDataset(batch_encodings, [0]*len(batch_texts))  # Dummy labels

    batch_predictions = trainer.predict(batch_dataset)
    batch_preds = np.argmax(batch_predictions.predictions, axis=-1)
    all_preds.extend(batch_preds)

elapsed_time = time.time() - start_time
print(f"Processing completed in {elapsed_time:.2f} seconds")

# Add predictions to the dataframe
clean_sampleset2000['predicted_sentiment'] = all_preds

# Save the dataframe with predictions
clean_sampleset2000.to_csv('C:/Users/Devanand J/Desktop/Final/Subset2000/GPT_clean_sampleset2000_overallpredictions.csv', index=False)

# Display the dataframe with predictions
import ace_tools as tools; tools.display_dataframe_to_user(name="clean_sampleset2000_with_predictions", dataframe=clean_sampleset2000)


Processing batches:   0%|          | 0/125 [00:00<?, ?it/s]

Processing batches:   1%|          | 1/125 [00:01<02:23,  1.16s/it]

Processing batches:   2%|▏         | 2/125 [00:01<01:26,  1.43it/s]

Processing batches:   2%|▏         | 3/125 [00:02<01:20,  1.52it/s]

Processing batches:   3%|▎         | 4/125 [00:03<01:29,  1.35it/s]

...

Processing batches:  98%|█████████▊| 123/125 [01:35<00:01,  1.54it/s]

Processing batches:  99%|█████████▉| 124/125 [01:36<00:00,  1.35it/s]

Processing batches: 100%|██████████| 125/125 [01:37<00:00,  1.28it/s]

Processing completed in 97.46 seconds


ModuleNotFoundError: No module named 'ace_tools'